
# 🧪 Prompting Playground — LangChain (Hands‑On)

**Last generated:** 2025-11-01 03:13:37

This notebook is a *hands-on* tour of practical prompting patterns using LangChain:
1. Basic `PromptTemplate`
2. `ChatPromptTemplate` (system + user messages)
3. `FewShotPromptTemplate`
4. Structured outputs with `PydanticOutputParser` (JSON)
5. RAG prompt patterns with `create_stuff_documents_chain`
6. (Optional) Map‑Reduce style document chain
7. Exercises

> 💡 Tip: Run this notebook in **Google Colab** or a local venv with Python 3.10+.


In [ ]:

# ✅ Setup (Uncomment if needed)
# If you're on Colab or a fresh environment, install the dependencies:
# !pip -q install -U langchain langchain-openai langchain-community pydantic pydantic-core tiktoken

import os

# --- Put your key here or set it in your environment before running ---
# os.environ["OPENAI_API_KEY"] = "sk-..."  # <- Prefer setting it in env, not here.

# Imports (LangChain 0.2+ / 0.3+ style)
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Optional: documents & chains
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain

# Utility for pretty printing
from pprint import pprint

# Create an LLM handle (adjust model to what you have access to)
# If you don't have gpt-4o, try "gpt-4o-mini" or "gpt-4.1-mini" or "gpt-4.1"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.environ.get("OPENAI_API_KEY", None))
str_parser = StrOutputParser()


## 1) Basic `PromptTemplate`

In [ ]:

# A simple single-message template (non-chat) for direct formatting
basic_prompt = PromptTemplate.from_template(
    "Explain the following concept in simple terms for a beginner:\n\nTopic: {topic}"
)

# 1) Inspect the formatted text (no model call yet)
formatted = basic_prompt.format(topic="Neural Networks")
print("---- Rendered Prompt ----\n", formatted)

# 2) Connect to LLM using LCEL pipe: prompt | llm | parser
basic_chain = basic_prompt | llm | str_parser

# 3) Invoke the chain
resp = basic_chain.invoke({"topic": "Neural Networks"})
print("\n---- LLM Response ----\n", resp)


## 2) `ChatPromptTemplate` (system + user messages)

In [ ]:

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a patient and friendly tutor. Respond concisely."),
    ("human", "Please explain {topic} with a simple metaphor and one bullet list of 3 key points.")
])

# View the rendered message list
rendered = chat_prompt.invoke({"topic": "Gradient Descent"})
print("---- Rendered Chat Messages ----")
for m in rendered.to_messages():
    print(m.type.upper() + ":", m.content)

# Build chain
chat_chain = chat_prompt | llm | str_parser
print("\n---- LLM Response ----\n", chat_chain.invoke({"topic": "Gradient Descent"}))


## 3) `FewShotPromptTemplate` (examples to steer the model)

In [ ]:

# Provide small labeled examples
examples = [
    {"term": "Overfitting", "explanation": "Model memorizes training data and performs poorly on new data."},
    {"term": "Regularization", "explanation": "A technique to discourage overly complex models and reduce overfitting."},
]

example_prompt = PromptTemplate.from_template("Term: {term}\nShort explanation: {explanation}")

fewshot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Now explain the term briefly:\nTerm: {query}",
    input_variables=["query"]
)

print("---- Rendered Few-Shot Prompt ----\n")
print(fewshot.format(query="Cross-Validation"))

fewshot_chain = fewshot | llm | str_parser
print("\n---- LLM Response ----\n", fewshot_chain.invoke({"query": "Cross-Validation"}))


## 4) Structured JSON output with `PydanticOutputParser`

In [ ]:

# Define a schema for the response
class Concept(BaseModel):
    term: str = Field(description="The technical term being defined")
    definition: str = Field(description="A concise definition in plain English")
    difficulty: int = Field(description="Difficulty from 1 (easy) to 5 (hard)")

parser = PydanticOutputParser(pydantic_object=Concept)

schema_instructions = parser.get_format_instructions()

structured_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a precise assistant that MUST output valid JSON matching the given schema."),
    ("human", "Term: {term}\n\nReturn JSON matching this schema:\n{format_instructions}")
]).partial(format_instructions=schema_instructions)

structured_chain = structured_prompt | llm | parser

result = structured_chain.invoke({"term": "Batch Normalization"})
print("---- Parsed Pydantic Object ----")
print(result)
print("\nAs dict:")
print(result.dict())


## 5) RAG prompt pattern — `create_stuff_documents_chain`

In [ ]:

# Build an instruction that accepts a {context} (documents) and a {question}
rag_prompt = ChatPromptTemplate.from_template(
    "You are a careful assistant. Answer the question using ONLY the provided context.\n"
    "If the answer is not in the context, say \"I don't know.\"\n\n"
    "<context>\n{context}\n</context>\n\n"
    "Question: {question}"
)

# Fake documents (you would use your retriever's results in practice)
docs = [
    Document(page_content="LangChain is a framework for building with LLMs. It helps with prompts, chains, and retrieval."),
    Document(page_content="The create_stuff_documents_chain function injects document text into a single prompt for the LLM."),
]

# Build a document chain that will render {context} from docs and pass to the LLM
doc_chain = create_stuff_documents_chain(llm, rag_prompt)

# 👉 How to see the final rendered prompt?
# The chain itself manages rendering, but we can emulate what it does by joining doc texts.
context_text = "\n\n".join(d.page_content for d in docs)
rendered_rag = rag_prompt.format(context=context_text, question="What is LangChain used for?")

print("---- Rendered RAG Prompt (preview) ----\n")
print(rendered_rag)

# Now invoke the actual chain (it will do the stuffing internally)
answer = doc_chain.invoke({"question": "What is LangChain used for?", "context": docs})
print("\n---- LLM Answer ----\n", answer)


## 6) (Optional) Map‑Reduce document chain

In [ ]:

# Map‑Reduce is useful for lots of long documents.
# In LangChain 0.2/0.3, the high-level factory for map-reduce may vary by version.
# Below is a pattern using two prompts and a manual reduce step (kept simple).

map_prompt = ChatPromptTemplate.from_template(
    "Summarize the following chunk concisely:\n\n{chunk}"
)
reduce_prompt = ChatPromptTemplate.from_template(
    "Combine the following partial summaries into one crisp summary:\n\n{summaries}"
)

chunks = [
    "Chunk A: LangChain supports prompts, LLMs, agents, and tools.",
    "Chunk B: Retrieval helps the model ground answers in external knowledge.",
    "Chunk C: Output parsers enforce structure such as JSON or Pydantic models."
]

# Map stage
map_chain = map_prompt | llm | str_parser
partials = [map_chain.invoke({"chunk": c}) for c in chunks]

# Reduce stage
reduce_chain = reduce_prompt | llm | str_parser
final_summary = reduce_chain.invoke({"summaries": "\n\n".join(partials)})

print("---- Map-Reduce Partial Summaries ----")
for i, p in enumerate(partials, 1):
    print(f"[{i}] {p}\n")
print("---- Final Combined Summary ----\n", final_summary)



## 7) Challenges / Exercises

1. **Guardrail**: Modify the structured prompt to include a `sources: List[str]` field and make the model cite anything it claims.
2. **Style Transfer**: Use `ChatPromptTemplate` to rewrite a paragraph in three different tones (formal, friendly, technical).
3. **RAG Swap**: Replace the fake `docs` with a real vectorstore retriever pipeline and feed the retrieved docs into the stuff chain.
4. **JSON Mode**: Try `JsonOutputParser` with a custom JSON schema (no Pydantic) and validate the result.
5. **Long Context**: Split a long PDF into chunks → map-summarize → reduce → answer a question.
